In [ ]:
from numba import njit, prange
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Rectangle
import time
import os
import 

In [23]:
try:
    import nfl_advanced_computation as adv
    HAS_ADVANCED_COMPUTATION = True
except ImportError:
    HAS_ADVANCED_COMPUTATION = False

# Global variables for simulation settings
QUARTERS = 4
QUARTER_LENGTH = 15 * 60  # 15 minutes in seconds
COMPUTE_INTENSITY = 500  # Increased from 50 to 500 for longer runtime

class NFLSimulator:
    def __init__(self, games_file="games.csv", plays_file="plays.csv"):
        """Initialize the NFL simulator with game and play data"""
        print("Loading data files...")
        self.games_df = pd.read_csv(games_file)
        self.plays_df = pd.read_csv(plays_file)
        
        # Get available teams
        self.teams = np.unique(np.concatenate([
            self.games_df['homeTeamAbbr'].unique(), 
            self.games_df['visitorTeamAbbr'].unique()
        ]))
        
        # Preprocess play data for faster simulation
        self._preprocess_data()
        
        print(f"Simulator ready with {len(self.teams)} teams available.")
    
    def _preprocess_data(self):
        """Preprocess play data for simulation"""
        # Create play type categories
        self.plays_df['playType'] = 'Other'
        
        # Categorize pass plays
        if 'isDropback' in self.plays_df.columns:
            pass_mask = (self.plays_df['isDropback'] == 1) & (~self.plays_df['passResult'].isna())
            self.plays_df.loc[pass_mask, 'playType'] = 'Pass'
        
        # Categorize run plays
        if 'isDropback' in self.plays_df.columns:
            run_mask = (self.plays_df['isDropback'] != 1) & (~self.plays_df['yardsGained'].isna())
            self.plays_df.loc[run_mask, 'playType'] = 'Run'
        
        # Categorize special teams plays
        if 'playDescription' in self.plays_df.columns:
            punt_mask = self.plays_df['playDescription'].str.contains('PUNT', na=False)
            fg_mask = self.plays_df['playDescription'].str.contains('FIELD GOAL', na=False)
            
            self.plays_df.loc[punt_mask, 'playType'] = 'Punt'
            self.plays_df.loc[fg_mask, 'playType'] = 'Field Goal'
        
        # Create play outcome dictionaries
        self.team_plays = {}
        for team in self.teams:
            team_df = self.plays_df[self.plays_df['possessionTeam'] == team]
            if len(team_df) > 0:
                self.team_plays[team] = {
                    'Pass': team_df[team_df['playType'] == 'Pass'],
                    'Run': team_df[team_df['playType'] == 'Run'],
                    'Punt': team_df[team_df['playType'] == 'Punt'],
                    'Field Goal': team_df[team_df['playType'] == 'Field Goal']
                }
    
    def display_available_teams(self):
        """Display all available teams for simulation"""
        teams = sorted(self.teams)
        print("\nAvailable Teams:")
        print(", ".join(teams))
        return teams
    
    def _intensive_computation(self, iterations=100):
        """Run intensive computation to make simulation take longer"""
        # Create large matrices
        a = np.random.normal(0, 1, (500, 500))  # Increased matrix size
        b = np.random.normal(0, 1, (500, 500))  # Increased matrix size
        
        # Run computationally intensive operations
        for _ in range(iterations):
            # Matrix multiplication (very intensive for large matrices)
            c = np.matmul(a, b)
            
            # Eigenvalue calculation (very computationally expensive)
            if _ % 10 == 0:  # Do this every 10 iterations to avoid too much slowdown
                try:
                    # Calculate eigenvalues of a subset of the matrix
                    subset_size = 100
                    subset = c[:subset_size, :subset_size]
                    eig_vals = np.linalg.eigvals(subset)
                    # Sort eigenvalues
                    eig_vals = np.sort(np.abs(eig_vals))
                except:
                    pass
            
            # Matrix operations
            d = np.linalg.norm(c, axis=1)
            e = np.sort(d)
            
            # More operations
            f = np.exp(c[:50, :50])
            g = np.sin(c[50:100, 50:100])
            
            # Calculate matrix determinants (very intensive)
            if _ % 20 == 0:  # Do this every 20 iterations
                try:
                    det_val = np.linalg.det(c[:100, :100])
                except:
                    pass
            
            # Update matrices slightly for next iteration
            a = 0.99 * a + 0.01 * np.random.normal(0, 1, (500, 500))
            b = 0.99 * b + 0.01 * np.random.normal(0, 1, (500, 500))
    
    def get_play(self, team, play_type, field_position, down, distance):
        """Get a play outcome based on historical data and intensive computation"""
        # Run computation to make this function take longer
        self._intensive_computation(COMPUTE_INTENSITY)
        
        # Use advanced computation if available
        if HAS_ADVANCED_COMPUTATION:
            # Create weather conditions (random for simulation)
            weather = {
                'wind_speed': np.random.randint(0, 20),
                'wind_direction': np.random.randint(0, 360),
                'temperature': np.random.randint(20, 95),
                'precipitation': np.random.uniform(0, 1.0),
                'field_condition': np.random.uniform(0.5, 1.0)
            }
            
            # Create game situation
            situation = {
                'down': down,
                'distance': distance,
                'field_position': field_position,
                'score_diff': 0  # Default neutral score
            }
            
            # Create team data (random for simulation)
            offense_team = {f"player_{i}": np.random.normal(0.7, 0.15, 10) for i in range(11)}
            defense_team = {f"player_{i}": np.random.normal(0.7, 0.15, 10) for i in range(11)}
            
            # Run advanced computations
            # 1. Simulate player tracking
            try:
                tracking_data = adv.simulate_player_tracking(22, 100)
                
                # Analyze tracking data to adjust play outcome
                # More computation to slow down simulation
                player_speeds = np.zeros(22)
                player_distances = np.zeros(22)
                for i in range(22):
                    # Calculate speeds and distances for each player
                    for t in range(1, 100):
                        # Calculate displacement between time steps
                        displacement = tracking_data[i, t, :] - tracking_data[i, t-1, :]
                        # Calculate speed
                        speed = np.linalg.norm(displacement) / 0.1  # 0.1 second time step
                        # Update maximum speed
                        player_speeds[i] = max(player_speeds[i], speed)
                        # Add to total distance
                        player_distances[i] += np.linalg.norm(displacement)
            except:
                # If tracking simulation fails, create dummy data
                tracking_data = None
                player_speeds = np.random.normal(5, 1, 22)
                player_distances = np.random.normal(20, 5, 22)
            
            # 2. Apply weather effects
            try:
                weather_adjustments = adv.simulate_weather_effects(field_position, play_type, weather)
            except:
                # If weather simulation fails, use default adjustments
                weather_adjustments = {
                    'yards': 0,
                    'accuracy': 0,
                    'turnover_risk': 0,
                    'success_probability': 0
                }
            
            # 3. Create player matchups
            try:
                matchup_results = adv.create_matchup_matrix(offense_team, defense_team, play_type, situation)
            except:
                # If matchup simulation fails, use default results
                matchup_results = {
                    'success_probability': 0.5,
                    'expected_yards': 5,
                    'turnover_risk': 0.03,
                    'advantage': 0
                }
                
            # Use advanced computations to determine play outcome
            base_yards = matchup_results['expected_yards']
            
            # Add randomness to yards
            yards = base_yards + np.random.normal(0, 3)
            
            # Apply weather adjustments
            yards += weather_adjustments['yards']
            
            # Adjust for field position (harder to gain yards near goal line)
            if field_position > 80:
                yards *= 0.7  # Reduce yards in red zone
            
            # Determine if turnover occurs
            turnover_probability = matchup_results['turnover_risk'] + weather_adjustments['turnover_risk']
            turnover = np.random.random() < turnover_probability
            
            # Determine if touchdown occurs
            if field_position + yards >= 100:
                touchdown = True
                yards = 100 - field_position  # Adjust yards to reach end zone
            else:
                touchdown = False
            
            # Determine if field goal is successful
            if play_type == 'Field Goal':
                fg_distance = 100 - field_position + 17  # Add 17 yards for end zone + snap
                base_fg_probability = 1.0 - (fg_distance / 65)  # Base probability by distance
                adjusted_fg_probability = base_fg_probability + weather_adjustments['success_probability']
                field_goal = np.random.random() < adjusted_fg_probability
            else:
                field_goal = False
            
            # Create play description
            if touchdown:
                description = f"{play_type} play for TOUCHDOWN!"
            elif field_goal:
                description = "FIELD GOAL IS GOOD"
            elif turnover:
                if play_type == 'Pass':
                    description = "Pass INTERCEPTED"
                else:
                    description = f"{play_type} play - FUMBLE"
            else:
                description = f"{play_type} play for {yards:.1f} yards"
            
            # Estimate play time (in seconds)
            play_time = np.random.randint(25, 40)
            
            return {
                'yards': yards,
                'time': play_time,
                'turnover': turnover,
                'touchdown': touchdown,
                'field_goal': field_goal,
                'description': description
            }
        
        # If no advanced computation, use the original method
        # If no data available for this team/play type, return default
        if team not in self.team_plays or play_type not in self.team_plays[team]:
            return {
                'yards': np.random.normal(5, 10),
                'time': np.random.randint(25, 40),
                'turnover': np.random.random() < 0.05,
                'touchdown': np.random.random() < 0.05,
                'field_goal': play_type == 'Field Goal' and np.random.random() < 0.7,
                'description': f"{play_type} play (simulated)"
            }
        
        # Get play data for this team/play type
        plays = self.team_plays[team][play_type]
        
        if len(plays) == 0:
            # No plays of this type, return default
            return {
                'yards': np.random.normal(5, 10),
                'time': np.random.randint(25, 40),
                'turnover': np.random.random() < 0.05,
                'touchdown': np.random.random() < 0.05,
                'field_goal': play_type == 'Field Goal' and np.random.random() < 0.7,
                'description': f"{play_type} play (no data)"
            }
        
        # Select a random play
        play = plays.sample(1).iloc[0]
        
        # Get yards gained (default to 0 if not available)
        yards = play['yardsGained'] if 'yardsGained' in play and not pd.isna(play['yardsGained']) else 0
        
        # Add some randomness to yards gained
        yards = yards + np.random.normal(0, yards/4 if yards > 0 else 2)
        
        # Check for touchdown based on field position
        touchdown = False
        if field_position + yards >= 100:
            touchdown = True
            yards = 100 - field_position
        
        # Check for turnover
        turnover = False
        if 'playDescription' in play:
            desc = str(play['playDescription']).upper()
            turnover = 'INTERCEPTION' in desc or 'FUMBLE' in desc
        
        # Check for field goal
        field_goal = False
        if play_type == 'Field Goal' and 'playDescription' in play:
            desc = str(play['playDescription']).upper()
            field_goal = 'FIELD GOAL IS GOOD' in desc
            
            # Adjust field goal probability based on field position
            if field_position >= 65:  # 35 yard line or closer
                field_goal = np.random.random() < 0.9  # 90% probability
            elif field_position >= 55:  # 45 yard line or closer
                field_goal = np.random.random() < 0.7  # 70% probability
            else:
                field_goal = np.random.random() < 0.4  # 40% probability
        
        # Create a play description
        if touchdown:
            description = f"{play_type} play for TOUCHDOWN!"
        elif field_goal:
            description = "FIELD GOAL IS GOOD"
        elif turnover:
            if play_type == 'Pass':
                description = "Pass INTERCEPTED"
            else:
                description = f"{play_type} play - FUMBLE"
        else:
            description = f"{play_type} play for {yards:.1f} yards"
        
        # Estimate play time
        play_time = np.random.randint(25, 40)
        
        return {
            'yards': yards,
            'time': play_time,
            'turnover': turnover,
            'touchdown': touchdown,
            'field_goal': field_goal,
            'description': description
        }
    
    def choose_play_type(self, down, distance, field_position, score_diff, time_remaining):
        """Choose play type based on game situation"""
        # Base probabilities
        pass_prob = 0.6
        run_prob = 0.4
        fg_prob = 0.0
        punt_prob = 0.0
        
        # Adjust based on down
        if down == 3:
            # More likely to pass on 3rd down
            pass_prob += 0.2
            run_prob -= 0.2
        elif down == 4:
            # 4th down decision making
            if field_position > 65:  # Inside opponent's 35
                # Field goal territory
                fg_prob = 0.8
                pass_prob = 0.1
                run_prob = 0.1
                punt_prob = 0.0
            elif field_position > 50:  # Between 50 and opponent's 35
                # Go for it or punt
                if distance <= 2:  # Short yardage
                    pass_prob = 0.3
                    run_prob = 0.4
                    punt_prob = 0.3
                else:
                    punt_prob = 0.9
                    pass_prob = 0.05
                    run_prob = 0.05
            else:  # Own territory
                # Likely punt
                punt_prob = 0.95
                pass_prob = 0.025
                run_prob = 0.025
        
        # Adjust for time remaining
        if time_remaining < 120:  # Last 2 minutes
            if score_diff < 0:  # Losing
                pass_prob += 0.2  # More passes when behind late
                run_prob -= 0.2
            elif score_diff > 14:  # Winning big
                run_prob += 0.2  # More runs when ahead late
                pass_prob -= 0.2
        
        # Normalize probabilities
        total = pass_prob + run_prob + fg_prob + punt_prob
        pass_prob /= total
        run_prob /= total
        fg_prob /= total
        punt_prob /= total
        
        # Choose play type
        r = np.random.random()
        if r < pass_prob:
            return 'Pass'
        elif r < pass_prob + run_prob:
            return 'Run'
        elif r < pass_prob + run_prob + fg_prob:
            return 'Field Goal'
        else:
            return 'Punt'
    
    def simulate_game(self, team1, team2):
        """Simulate a full game between two teams"""
        start_time = time.time()
        print(f"\nSimulating {team1} vs {team2}...")
        
        # Game state initialization
        score = {team1: 0, team2: 0}
        possession = np.random.choice([team1, team2])  # Random team starts with ball
        field_position = 25  # Start at own 25 yard line
        down = 1
        distance = 10
        time_remaining = QUARTERS * QUARTER_LENGTH
        quarter = 1
        
        # Game statistics
        stats = {
            team1: {'total_yards': 0, 'pass_yards': 0, 'rush_yards': 0, 'turnovers': 0},
            team2: {'total_yards': 0, 'pass_yards': 0, 'rush_yards': 0, 'turnovers': 0}
        }
        
        # Drive tracking
        drive_summary = []
        current_drive = {
            'team': possession,
            'start_position': field_position,
            'plays': [],
            'result': None,
            'quarter': quarter,
            'time': f"{time_remaining // 60:02d}:{time_remaining % 60:02d}"
        }
        
        # Team colors
        team_colors = {team1: 'red', team2: 'blue'}
        
        # Game loop
        while time_remaining > 0:
            # Get the defensive team
            defense = team2 if possession == team1 else team1
            
            # Calculate score differential from possession team perspective
            score_diff = score[possession] - score[defense]
            
            # Choose play type
            play_type = self.choose_play_type(
                down, distance, field_position, score_diff, time_remaining
            )
            
            # Get play result
            play = self.get_play(possession, play_type, field_position, down, distance)
            
            # Apply play result
            yards_gained = play['yards']
            play_time = play['time']
            
            # Update statistics
            stats[possession]['total_yards'] += yards_gained
            if play_type == 'Pass':
                stats[possession]['pass_yards'] += yards_gained
            elif play_type == 'Run':
                stats[possession]['rush_yards'] += yards_gained
            
            # Calculate time used
            time_remaining -= play_time
            
            # Update quarter if needed
            new_quarter = min(4, 1 + (QUARTERS * QUARTER_LENGTH - time_remaining) // QUARTER_LENGTH)
            quarter_change = new_quarter > quarter
            quarter = new_quarter
            
            # Format game clock
            minutes = time_remaining // 60
            seconds = time_remaining % 60
            clock_display = f"{int(minutes):02}:{int(seconds):02}"
            
            # Add play to current drive
            current_drive['plays'].append({
                'type': play_type,
                'yards': yards_gained,
                'description': play['description'],
                'down': down,
                'distance': distance,
                'field_position': field_position
            })
            
            # Handle scoring plays
            if play['touchdown']:
                score[possession] += 7  # TD + PAT for simplicity
                current_drive['result'] = 'TOUCHDOWN'
                
                # Complete the drive
                current_drive['end_position'] = 100
                current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                drive_summary.append(current_drive)
                
                # Print drive summary
                print(f"\nDrive Summary - {possession} TOUCHDOWN!")
                print(f"Started at: {current_drive['start_position']} yard line")
                print(f"Plays: {len(current_drive['plays'])}")
                print(f"Yards: {current_drive['yards']}")
                print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                print(f"Score: {possession} {score[possession]}, {defense} {score[defense]}")
                
                # Start new drive
                possession = defense
                field_position = 25
                down = 1
                distance = 10
                current_drive = {
                    'team': possession,
                    'start_position': field_position,
                    'plays': [],
                    'result': None,
                    'quarter': quarter,
                    'time': clock_display
                }
                continue
                
            elif play['field_goal']:
                score[possession] += 3
                current_drive['result'] = 'FIELD GOAL'
                
                # Complete the drive
                current_drive['end_position'] = field_position
                current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                drive_summary.append(current_drive)
                
                # Print drive summary
                print(f"\nDrive Summary - {possession} FIELD GOAL!")
                print(f"Started at: {current_drive['start_position']} yard line")
                print(f"Plays: {len(current_drive['plays'])}")
                print(f"Yards: {current_drive['yards']}")
                print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                print(f"Score: {possession} {score[possession]}, {defense} {score[defense]}")
                
                # Start new drive
                possession = defense
                field_position = 25
                down = 1
                distance = 10
                current_drive = {
                    'team': possession,
                    'start_position': field_position,
                    'plays': [],
                    'result': None,
                    'quarter': quarter,
                    'time': clock_display
                }
                continue
            
            # Handle turnovers
            if play['turnover']:
                stats[possession]['turnovers'] += 1
                current_drive['result'] = 'TURNOVER'
                
                # Complete the drive
                current_drive['end_position'] = field_position + yards_gained
                current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                drive_summary.append(current_drive)
                
                # Print drive summary
                print(f"\nDrive Summary - {possession} TURNOVER!")
                print(f"Started at: {current_drive['start_position']} yard line")
                print(f"Plays: {len(current_drive['plays'])}")
                print(f"Yards: {current_drive['yards']}")
                print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                print(f"Final play: {play['description']}")
                
                # Switch possession
                possession = defense
                field_position = 100 - (field_position + yards_gained)  # Flip field
                field_position = max(0, min(100, field_position))  # Keep in bounds
                down = 1
                distance = 10
                
                # Start new drive
                current_drive = {
                    'team': possession,
                    'start_position': field_position,
                    'plays': [],
                    'result': None,
                    'quarter': quarter,
                    'time': clock_display
                }
                continue
            
            # Handle punts
            if play_type == 'Punt':
                # Punt distance based on field position
                punt_distance = np.random.normal(45, 8)
                current_drive['result'] = 'PUNT'
                
                # Complete the drive
                current_drive['end_position'] = field_position
                current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                drive_summary.append(current_drive)
                
                # Print drive summary
                print(f"\nDrive Summary - {possession} PUNT!")
                print(f"Started at: {current_drive['start_position']} yard line")
                print(f"Plays: {len(current_drive['plays'])}")
                print(f"Yards: {current_drive['yards']}")
                print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                print(f"Punt distance: {punt_distance:.1f} yards")
                
                # Switch possession
                possession = defense
                field_position = max(0, min(100, 100 - (field_position + punt_distance)))
                down = 1
                distance = 10
                
                # Start new drive
                current_drive = {
                    'team': possession,
                    'start_position': field_position,
                    'plays': [],
                    'result': None,
                    'quarter': quarter,
                    'time': clock_display
                }
                continue
            
            # Update field position
            field_position += yards_gained
            
            # Check for first down
            if yards_gained >= distance:
                down = 1
                distance = 10
                if field_position > 100:
                    field_position = 100  # Cap at 100 (goal line)
            else:
                down += 1
                distance -= yards_gained
                if field_position > 100:
                    # Crossed goal line
                    score[possession] += 7  # TD + PAT
                    current_drive['result'] = 'TOUCHDOWN'
                    
                    # Complete the drive
                    current_drive['end_position'] = 100
                    current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                    drive_summary.append(current_drive)
                    
                    # Print drive summary
                    print(f"\nDrive Summary - {possession} TOUCHDOWN!")
                    print(f"Started at: {current_drive['start_position']} yard line")
                    print(f"Plays: {len(current_drive['plays'])}")
                    print(f"Yards: {current_drive['yards']}")
                    print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                    print(f"Score: {possession} {score[possession]}, {defense} {score[defense]}")
                    
                    # Switch possession
                    possession = defense
                    field_position = 25
                    down = 1
                    distance = 10
                    
                    # Start new drive
                    current_drive = {
                        'team': possession,
                        'start_position': field_position,
                        'plays': [],
                        'result': None,
                        'quarter': quarter,
                        'time': clock_display
                    }
                    continue
                
                if down > 4:
                    # Turnover on downs
                    current_drive['result'] = 'TURNOVER ON DOWNS'
                    
                    # Complete the drive
                    current_drive['end_position'] = field_position
                    current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                    drive_summary.append(current_drive)
                    
                    # Print drive summary
                    print(f"\nDrive Summary - {possession} TURNOVER ON DOWNS!")
                    print(f"Started at: {current_drive['start_position']} yard line")
                    print(f"Plays: {len(current_drive['plays'])}")
                    print(f"Yards: {current_drive['yards']}")
                    print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                    print(f"Failed to convert: {down-1} & {distance + yards_gained}")
                    
                    # Switch possession
                    possession = defense
                    field_position = 100 - field_position  # Flip field
                    down = 1
                    distance = 10
                    
                    # Start new drive
                    current_drive = {
                        'team': possession,
                        'start_position': field_position,
                        'plays': [],
                        'result': None,
                        'quarter': quarter,
                        'time': clock_display
                    }
                    continue
            
            # Keep field position in bounds
            field_position = max(0, min(100, field_position))
            
            # Handle end of quarter
            if quarter_change:
                print(f"\nEnd of Quarter {quarter-1}")
                print(f"Score: {team1} {score[team1]}, {team2} {score[team2]}")
                
                # If it's halftime, add a special message
                if quarter == 3:
                    print("HALFTIME")
                    
                    # Complete the current drive and start a new one after halftime
                    if current_drive['plays']:
                        current_drive['result'] = 'END OF HALF'
                        current_drive['end_position'] = field_position
                        current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                        drive_summary.append(current_drive)
                        
                        # Start a new drive in the second half
                        current_drive = {
                            'team': possession,
                            'start_position': field_position,
                            'plays': [],
                            'result': None,
                            'quarter': quarter,
                            'time': clock_display
                        }
        
        # Game over
        elapsed = time.time() - start_time
        print(f"\nSimulation completed in {elapsed:.2f} seconds")
        
        # If there's an incomplete drive at the end of the game, record it
        if current_drive['plays']:
            current_drive['result'] = 'END OF GAME'
            current_drive['end_position'] = field_position
            current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
            drive_summary.append(current_drive)
        
        # Print final game statistics
        print("\n📊 Game Statistics:")
        print(f"{team1.ljust(4)} | {team2.ljust(4)}")
        print("-" * 30)
        print(f"Total Yards: {stats[team1]['total_yards']:.0f} | {stats[team2]['total_yards']:.0f}")
        print(f"Pass Yards: {stats[team1]['pass_yards']:.0f} | {stats[team2]['pass_yards']:.0f}")
        print(f"Rush Yards: {stats[team1]['rush_yards']:.0f} | {stats[team2]['rush_yards']:.0f}")
        print(f"Turnovers: {stats[team1]['turnovers']} | {stats[team2]['turnovers']}")
        print(f"Score: {score}")
        
        # Print drive summary
        print("\n🏈 Drive Summary:")
        
        team1_drives = [d for d in drive_summary if d['team'] == team1]
        team2_drives = [d for d in drive_summary if d['team'] == team2]
        
        print(f"\n{team1} Drives: {len(team1_drives)}")
        td_drives = len([d for d in team1_drives if d['result'] == 'TOUCHDOWN'])
        fg_drives = len([d for d in team1_drives if d['result'] == 'FIELD GOAL'])
        punt_drives = len([d for d in team1_drives if d['result'] == 'PUNT'])
        turnover_drives = len([d for d in team1_drives if d['result'] in ['TURNOVER', 'TURNOVER ON DOWNS']])
        
        print(f"  Touchdowns: {td_drives}")
        print(f"  Field Goals: {fg_drives}")
        print(f"  Punts: {punt_drives}")
        print(f"  Turnovers: {turnover_drives}")
        
        print(f"\n{team2} Drives: {len(team2_drives)}")
        td_drives = len([d for d in team2_drives if d['result'] == 'TOUCHDOWN'])
        fg_drives = len([d for d in team2_drives if d['result'] == 'FIELD GOAL'])
        punt_drives = len([d for d in team2_drives if d['result'] == 'PUNT'])
        turnover_drives = len([d for d in team2_drives if d['result'] in ['TURNOVER', 'TURNOVER ON DOWNS']])
        
        print(f"  Touchdowns: {td_drives}")
        print(f"  Field Goals: {fg_drives}")
        print(f"  Punts: {punt_drives}")
        print(f"  Turnovers: {turnover_drives}")
        
        return {
            'team1': team1,
            'team2': team2,
            'score': score,
            'drive_summary': drive_summary,
            'stats': stats
        }

In [41]:
simulator = NFLSimulator()
simulator.simulate_game("DEN","DET")

Loading data files...
Simulator ready with 32 teams available.

Simulating DEN vs DET...

Drive Summary - DEN TOUCHDOWN!
Started at: 25 yard line
Plays: 4
Yards: 75
Time: 60:00 in Q1
Score: DEN 7, DET 0

Drive Summary - DET PUNT!
Started at: 25 yard line
Plays: 5
Yards: 18.84559697426893
Time: 57:50 in Q1
Punt distance: 58.8 yards

Drive Summary - DEN TURNOVER!
Started at: 0 yard line
Plays: 3
Yards: 9.111102655841197
Time: 55:27 in Q1
Final play: Pass INTERCEPTED

Drive Summary - DET TOUCHDOWN!
Started at: 90.8888973441588 yard line
Plays: 1
Yards: 9.111102655841194
Time: 53:53 in Q1
Score: DET 7, DEN 7

Drive Summary - DEN PUNT!
Started at: 25 yard line
Plays: 9
Yards: 27.82064715072731
Time: 53:24 in Q1
Punt distance: 47.3 yards

End of Quarter 1
Score: DEN 7, DET 7

Drive Summary - DET TOUCHDOWN!
Started at: 0 yard line
Plays: 14
Yards: 100
Time: 48:55 in Q1
Score: DET 14, DEN 7

Drive Summary - DEN TOUCHDOWN!
Started at: 25 yard line
Plays: 12
Yards: 75
Time: 41:31 in Q2
Score: DE

{'team1': 'DEN',
 'team2': 'DET',
 'score': {'DEN': 24, 'DET': 21},
 'drive_summary': [{'team': 'DEN',
   'start_position': 25,
   'plays': [{'type': 'Pass',
     'yards': -1.0580427454790402,
     'description': 'Pass play for -1.1 yards',
     'down': 1,
     'distance': 10,
     'field_position': 25},
    {'type': 'Pass',
     'yards': 5.503680383151911,
     'description': 'Pass play for 5.5 yards',
     'down': 2,
     'distance': 11.05804274547904,
     'field_position': 23.94195725452096},
    {'type': 'Run',
     'yards': 4.210651801192061,
     'description': 'Run play for 4.2 yards',
     'down': 3,
     'distance': 5.554362362327129,
     'field_position': 29.445637637672874},
    {'type': 'Punt',
     'yards': 3.3491281373758524,
     'description': 'Punt play (no data)',
     'down': 4,
     'distance': 1.3437105611350688,
     'field_position': 33.65628943886493}],
   'result': 'TOUCHDOWN',
   'quarter': 1,
   'time': '60:00',
   'end_position': 100,
   'yards': 75},
  {'

In [46]:
try:
    import nfl_advanced_computation as adv
    HAS_ADVANCED_COMPUTATION = True
except ImportError:
    HAS_ADVANCED_COMPUTATION = False

# Global simulation settings
QUARTERS = 4
QUARTER_LENGTH = 15 * 60
COMPUTE_INTENSITY = 500

from numba import njit, prange

@njit(parallel=True, fastmath=True)
def rowwise_norm(mat):
    nrows = mat.shape[0]
    result = np.empty(nrows)
    for i in prange(nrows):
        total = 0.0
        for j in range(mat.shape[1]):
            total += mat[i, j] ** 2
        result[i] = np.sqrt(total)
    return result

@njit(fastmath=True)
def run_numba_intensive_computation(iterations, a, b):
    for _ in range(iterations):
        c = a @ b
        d = rowwise_norm(c)
        e = np.sort(d)
        f = np.exp(c[:50, :50])
        g = np.sin(c[50:100, 50:100])
        noise = np.random.normal(0, 1, a.shape)
        a = 0.99 * a + 0.01 * noise
        b = 0.99 * b + 0.01 * noise
    return a, b

class NFLSimulator:
    def __init__(self, games_file="games.csv", plays_file="plays.csv"):
        print("Loading data files...")
        self.games_df = pd.read_csv(games_file)
        self.plays_df = pd.read_csv(plays_file)
        self.teams = np.unique(np.concatenate([
            self.games_df['homeTeamAbbr'].unique(), 
            self.games_df['visitorTeamAbbr'].unique()
        ]))
        self._preprocess_data()
        print(f"Simulator ready with {len(self.teams)} teams available.")

    def _preprocess_data(self):
        self.plays_df['playType'] = 'Other'
        if 'isDropback' in self.plays_df.columns:
            pass_mask = (self.plays_df['isDropback'] == 1) & (~self.plays_df['passResult'].isna())
            self.plays_df.loc[pass_mask, 'playType'] = 'Pass'
            run_mask = (self.plays_df['isDropback'] != 1) & (~self.plays_df['yardsGained'].isna())
            self.plays_df.loc[run_mask, 'playType'] = 'Run'
        if 'playDescription' in self.plays_df.columns:
            punt_mask = self.plays_df['playDescription'].str.contains('PUNT', na=False)
            fg_mask = self.plays_df['playDescription'].str.contains('FIELD GOAL', na=False)
            self.plays_df.loc[punt_mask, 'playType'] = 'Punt'
            self.plays_df.loc[fg_mask, 'playType'] = 'Field Goal'
        self.team_plays = {}
        for team in self.teams:
            team_df = self.plays_df[self.plays_df['possessionTeam'] == team]
            if len(team_df) > 0:
                self.team_plays[team] = {
                    'Pass': team_df[team_df['playType'] == 'Pass'],
                    'Run': team_df[team_df['playType'] == 'Run'],
                    'Punt': team_df[team_df['playType'] == 'Punt'],
                    'Field Goal': team_df[team_df['playType'] == 'Field Goal']
                }

    def _intensive_computation(self, iterations=100):
        a = np.random.normal(0, 1, (500, 500))
        b = np.random.normal(0, 1, (500, 500))
        run_numba_intensive_computation(iterations, a, b)

    def display_available_teams(self):
        teams = sorted(self.teams)
        print("\nAvailable Teams:")
        print(", ".join(teams))
        return teams

    def get_play(self, team, play_type, field_position, down, distance):
        """Get a play outcome based on historical data and intensive computation"""
        # Run computation to make this function take longer
        self._intensive_computation(COMPUTE_INTENSITY)
        
        # Use advanced computation if available
        if HAS_ADVANCED_COMPUTATION:
            # Create weather conditions (random for simulation)
            weather = {
                'wind_speed': np.random.randint(0, 20),
                'wind_direction': np.random.randint(0, 360),
                'temperature': np.random.randint(20, 95),
                'precipitation': np.random.uniform(0, 1.0),
                'field_condition': np.random.uniform(0.5, 1.0)
            }
            
            # Create game situation
            situation = {
                'down': down,
                'distance': distance,
                'field_position': field_position,
                'score_diff': 0  # Default neutral score
            }
            
            # Create team data (random for simulation)
            offense_team = {f"player_{i}": np.random.normal(0.7, 0.15, 10) for i in range(11)}
            defense_team = {f"player_{i}": np.random.normal(0.7, 0.15, 10) for i in range(11)}
            
            # Run advanced computations
            # 1. Simulate player tracking
            try:
                tracking_data = adv.simulate_player_tracking(22, 100)
                
                # Analyze tracking data to adjust play outcome
                # More computation to slow down simulation
                player_speeds = np.zeros(22)
                player_distances = np.zeros(22)
                for i in range(22):
                    # Calculate speeds and distances for each player
                    for t in range(1, 100):
                        # Calculate displacement between time steps
                        displacement = tracking_data[i, t, :] - tracking_data[i, t-1, :]
                        # Calculate speed
                        speed = np.linalg.norm(displacement) / 0.1  # 0.1 second time step
                        # Update maximum speed
                        player_speeds[i] = max(player_speeds[i], speed)
                        # Add to total distance
                        player_distances[i] += np.linalg.norm(displacement)
            except:
                # If tracking simulation fails, create dummy data
                tracking_data = None
                player_speeds = np.random.normal(5, 1, 22)
                player_distances = np.random.normal(20, 5, 22)
            
            # 2. Apply weather effects
            try:
                weather_adjustments = adv.simulate_weather_effects(field_position, play_type, weather)
            except:
                # If weather simulation fails, use default adjustments
                weather_adjustments = {
                    'yards': 0,
                    'accuracy': 0,
                    'turnover_risk': 0,
                    'success_probability': 0
                }
            
            # 3. Create player matchups
            try:
                matchup_results = adv.create_matchup_matrix(offense_team, defense_team, play_type, situation)
            except:
                # If matchup simulation fails, use default results
                matchup_results = {
                    'success_probability': 0.5,
                    'expected_yards': 5,
                    'turnover_risk': 0.03,
                    'advantage': 0
                }
                
            # Use advanced computations to determine play outcome
            base_yards = matchup_results['expected_yards']
            
            # Add randomness to yards
            yards = base_yards + np.random.normal(0, 3)
            
            # Apply weather adjustments
            yards += weather_adjustments['yards']
            
            # Adjust for field position (harder to gain yards near goal line)
            if field_position > 80:
                yards *= 0.7  # Reduce yards in red zone
            
            # Determine if turnover occurs
            turnover_probability = matchup_results['turnover_risk'] + weather_adjustments['turnover_risk']
            turnover = np.random.random() < turnover_probability
            
            # Determine if touchdown occurs
            if field_position + yards >= 100:
                touchdown = True
                yards = 100 - field_position  # Adjust yards to reach end zone
            else:
                touchdown = False
            
            # Determine if field goal is successful
            if play_type == 'Field Goal':
                fg_distance = 100 - field_position + 17  # Add 17 yards for end zone + snap
                base_fg_probability = 1.0 - (fg_distance / 65)  # Base probability by distance
                adjusted_fg_probability = base_fg_probability + weather_adjustments['success_probability']
                field_goal = np.random.random() < adjusted_fg_probability
            else:
                field_goal = False
            
            # Create play description
            if touchdown:
                description = f"{play_type} play for TOUCHDOWN!"
            elif field_goal:
                description = "FIELD GOAL IS GOOD"
            elif turnover:
                if play_type == 'Pass':
                    description = "Pass INTERCEPTED"
                else:
                    description = f"{play_type} play - FUMBLE"
            else:
                description = f"{play_type} play for {yards:.1f} yards"
            
            # Estimate play time (in seconds)
            play_time = np.random.randint(25, 40)
            
            return {
                'yards': yards,
                'time': play_time,
                'turnover': turnover,
                'touchdown': touchdown,
                'field_goal': field_goal,
                'description': description
            }
        
        # If no advanced computation, use the original method
        # If no data available for this team/play type, return default
        if team not in self.team_plays or play_type not in self.team_plays[team]:
            return {
                'yards': np.random.normal(5, 10),
                'time': np.random.randint(25, 40),
                'turnover': np.random.random() < 0.05,
                'touchdown': np.random.random() < 0.05,
                'field_goal': play_type == 'Field Goal' and np.random.random() < 0.7,
                'description': f"{play_type} play (simulated)"
            }
        
        # Get play data for this team/play type
        plays = self.team_plays[team][play_type]
        
        if len(plays) == 0:
            # No plays of this type, return default
            return {
                'yards': np.random.normal(5, 10),
                'time': np.random.randint(25, 40),
                'turnover': np.random.random() < 0.05,
                'touchdown': np.random.random() < 0.05,
                'field_goal': play_type == 'Field Goal' and np.random.random() < 0.7,
                'description': f"{play_type} play (no data)"
            }
        
        # Select a random play
        play = plays.sample(1).iloc[0]
        
        # Get yards gained (default to 0 if not available)
        yards = play['yardsGained'] if 'yardsGained' in play and not pd.isna(play['yardsGained']) else 0
        
        # Add some randomness to yards gained
        yards = yards + np.random.normal(0, yards/4 if yards > 0 else 2)
        
        # Check for touchdown based on field position
        touchdown = False
        if field_position + yards >= 100:
            touchdown = True
            yards = 100 - field_position
        
        # Check for turnover
        turnover = False
        if 'playDescription' in play:
            desc = str(play['playDescription']).upper()
            turnover = 'INTERCEPTION' in desc or 'FUMBLE' in desc
        
        # Check for field goal
        field_goal = False
        if play_type == 'Field Goal' and 'playDescription' in play:
            desc = str(play['playDescription']).upper()
            field_goal = 'FIELD GOAL IS GOOD' in desc
            
            # Adjust field goal probability based on field position
            if field_position >= 65:  # 35 yard line or closer
                field_goal = np.random.random() < 0.9  # 90% probability
            elif field_position >= 55:  # 45 yard line or closer
                field_goal = np.random.random() < 0.7  # 70% probability
            else:
                field_goal = np.random.random() < 0.4  # 40% probability
        
        # Create a play description
        if touchdown:
            description = f"{play_type} play for TOUCHDOWN!"
        elif field_goal:
            description = "FIELD GOAL IS GOOD"
        elif turnover:
            if play_type == 'Pass':
                description = "Pass INTERCEPTED"
            else:
                description = f"{play_type} play - FUMBLE"
        else:
            description = f"{play_type} play for {yards:.1f} yards"
        
        # Estimate play time
        play_time = np.random.randint(25, 40)
        
        return {
            'yards': yards,
            'time': play_time,
            'turnover': turnover,
            'touchdown': touchdown,
            'field_goal': field_goal,
            'description': description
        }
    
    def choose_play_type(self, down, distance, field_position, score_diff, time_remaining):
        """Choose play type based on game situation"""
        # Base probabilities
        pass_prob = 0.6
        run_prob = 0.4
        fg_prob = 0.0
        punt_prob = 0.0
        
        # Adjust based on down
        if down == 3:
            # More likely to pass on 3rd down
            pass_prob += 0.2
            run_prob -= 0.2
        elif down == 4:
            # 4th down decision making
            if field_position > 65:  # Inside opponent's 35
                # Field goal territory
                fg_prob = 0.8
                pass_prob = 0.1
                run_prob = 0.1
                punt_prob = 0.0
            elif field_position > 50:  # Between 50 and opponent's 35
                # Go for it or punt
                if distance <= 2:  # Short yardage
                    pass_prob = 0.3
                    run_prob = 0.4
                    punt_prob = 0.3
                else:
                    punt_prob = 0.9
                    pass_prob = 0.05
                    run_prob = 0.05
            else:  # Own territory
                # Likely punt
                punt_prob = 0.95
                pass_prob = 0.025
                run_prob = 0.025
        
        # Adjust for time remaining
        if time_remaining < 120:  # Last 2 minutes
            if score_diff < 0:  # Losing
                pass_prob += 0.2  # More passes when behind late
                run_prob -= 0.2
            elif score_diff > 14:  # Winning big
                run_prob += 0.2  # More runs when ahead late
                pass_prob -= 0.2
        
        # Normalize probabilities
        total = pass_prob + run_prob + fg_prob + punt_prob
        pass_prob /= total
        run_prob /= total
        fg_prob /= total
        punt_prob /= total
        
        # Choose play type
        r = np.random.random()
        if r < pass_prob:
            return 'Pass'
        elif r < pass_prob + run_prob:
            return 'Run'
        elif r < pass_prob + run_prob + fg_prob:
            return 'Field Goal'
        else:
            return 'Punt'
    
    def simulate_game(self, team1, team2):
        """Simulate a full game between two teams"""
        start_time = time.time()
        print(f"\nSimulating {team1} vs {team2}...")
        
        # Game state initialization
        score = {team1: 0, team2: 0}
        possession = np.random.choice([team1, team2])  # Random team starts with ball
        field_position = 25  # Start at own 25 yard line
        down = 1
        distance = 10
        time_remaining = QUARTERS * QUARTER_LENGTH
        quarter = 1
        
        # Game statistics
        stats = {
            team1: {'total_yards': 0, 'pass_yards': 0, 'rush_yards': 0, 'turnovers': 0},
            team2: {'total_yards': 0, 'pass_yards': 0, 'rush_yards': 0, 'turnovers': 0}
        }
        
        # Drive tracking
        drive_summary = []
        current_drive = {
            'team': possession,
            'start_position': field_position,
            'plays': [],
            'result': None,
            'quarter': quarter,
            'time': f"{time_remaining // 60:02d}:{time_remaining % 60:02d}"
        }
        
        # Team colors
        team_colors = {team1: 'red', team2: 'blue'}
        
        # Game loop
        while time_remaining > 0:
            # Get the defensive team
            defense = team2 if possession == team1 else team1
            
            # Calculate score differential from possession team perspective
            score_diff = score[possession] - score[defense]
            
            # Choose play type
            play_type = self.choose_play_type(
                down, distance, field_position, score_diff, time_remaining
            )
            
            # Get play result
            play = self.get_play(possession, play_type, field_position, down, distance)
            
            # Apply play result
            yards_gained = play['yards']
            play_time = play['time']
            
            # Update statistics
            stats[possession]['total_yards'] += yards_gained
            if play_type == 'Pass':
                stats[possession]['pass_yards'] += yards_gained
            elif play_type == 'Run':
                stats[possession]['rush_yards'] += yards_gained
            
            # Calculate time used
            time_remaining -= play_time
            
            # Update quarter if needed
            new_quarter = min(4, 1 + (QUARTERS * QUARTER_LENGTH - time_remaining) // QUARTER_LENGTH)
            quarter_change = new_quarter > quarter
            quarter = new_quarter
            
            # Format game clock
            minutes = time_remaining // 60
            seconds = time_remaining % 60
            clock_display = f"{int(minutes):02}:{int(seconds):02}"
            
            # Add play to current drive
            current_drive['plays'].append({
                'type': play_type,
                'yards': yards_gained,
                'description': play['description'],
                'down': down,
                'distance': distance,
                'field_position': field_position
            })
            
            # Handle scoring plays
            if play['touchdown']:
                score[possession] += 7  # TD + PAT for simplicity
                current_drive['result'] = 'TOUCHDOWN'
                
                # Complete the drive
                current_drive['end_position'] = 100
                current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                drive_summary.append(current_drive)
                
                # Print drive summary
                print(f"\nDrive Summary - {possession} TOUCHDOWN!")
                print(f"Started at: {current_drive['start_position']} yard line")
                print(f"Plays: {len(current_drive['plays'])}")
                print(f"Yards: {current_drive['yards']}")
                print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                print(f"Score: {possession} {score[possession]}, {defense} {score[defense]}")
                
                # Start new drive
                possession = defense
                field_position = 25
                down = 1
                distance = 10
                current_drive = {
                    'team': possession,
                    'start_position': field_position,
                    'plays': [],
                    'result': None,
                    'quarter': quarter,
                    'time': clock_display
                }
                continue
                
            elif play['field_goal']:
                score[possession] += 3
                current_drive['result'] = 'FIELD GOAL'
                
                # Complete the drive
                current_drive['end_position'] = field_position
                current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                drive_summary.append(current_drive)
                
                # Print drive summary
                print(f"\nDrive Summary - {possession} FIELD GOAL!")
                print(f"Started at: {current_drive['start_position']} yard line")
                print(f"Plays: {len(current_drive['plays'])}")
                print(f"Yards: {current_drive['yards']}")
                print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                print(f"Score: {possession} {score[possession]}, {defense} {score[defense]}")
                
                # Start new drive
                possession = defense
                field_position = 25
                down = 1
                distance = 10
                current_drive = {
                    'team': possession,
                    'start_position': field_position,
                    'plays': [],
                    'result': None,
                    'quarter': quarter,
                    'time': clock_display
                }
                continue
            
            # Handle turnovers
            if play['turnover']:
                stats[possession]['turnovers'] += 1
                current_drive['result'] = 'TURNOVER'
                
                # Complete the drive
                current_drive['end_position'] = field_position + yards_gained
                current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                drive_summary.append(current_drive)
                
                # Print drive summary
                print(f"\nDrive Summary - {possession} TURNOVER!")
                print(f"Started at: {current_drive['start_position']} yard line")
                print(f"Plays: {len(current_drive['plays'])}")
                print(f"Yards: {current_drive['yards']}")
                print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                print(f"Final play: {play['description']}")
                
                # Switch possession
                possession = defense
                field_position = 100 - (field_position + yards_gained)  # Flip field
                field_position = max(0, min(100, field_position))  # Keep in bounds
                down = 1
                distance = 10
                
                # Start new drive
                current_drive = {
                    'team': possession,
                    'start_position': field_position,
                    'plays': [],
                    'result': None,
                    'quarter': quarter,
                    'time': clock_display
                }
                continue
            
            # Handle punts
            if play_type == 'Punt':
                # Punt distance based on field position
                punt_distance = np.random.normal(45, 8)
                current_drive['result'] = 'PUNT'
                
                # Complete the drive
                current_drive['end_position'] = field_position
                current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                drive_summary.append(current_drive)
                
                # Print drive summary
                print(f"\nDrive Summary - {possession} PUNT!")
                print(f"Started at: {current_drive['start_position']} yard line")
                print(f"Plays: {len(current_drive['plays'])}")
                print(f"Yards: {current_drive['yards']}")
                print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                print(f"Punt distance: {punt_distance:.1f} yards")
                
                # Switch possession
                possession = defense
                field_position = max(0, min(100, 100 - (field_position + punt_distance)))
                down = 1
                distance = 10
                
                # Start new drive
                current_drive = {
                    'team': possession,
                    'start_position': field_position,
                    'plays': [],
                    'result': None,
                    'quarter': quarter,
                    'time': clock_display
                }
                continue
            
            # Update field position
            field_position += yards_gained
            
            # Check for first down
            if yards_gained >= distance:
                down = 1
                distance = 10
                if field_position > 100:
                    field_position = 100  # Cap at 100 (goal line)
            else:
                down += 1
                distance -= yards_gained
                if field_position > 100:
                    # Crossed goal line
                    score[possession] += 7  # TD + PAT
                    current_drive['result'] = 'TOUCHDOWN'
                    
                    # Complete the drive
                    current_drive['end_position'] = 100
                    current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                    drive_summary.append(current_drive)
                    
                    # Print drive summary
                    print(f"\nDrive Summary - {possession} TOUCHDOWN!")
                    print(f"Started at: {current_drive['start_position']} yard line")
                    print(f"Plays: {len(current_drive['plays'])}")
                    print(f"Yards: {current_drive['yards']}")
                    print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                    print(f"Score: {possession} {score[possession]}, {defense} {score[defense]}")
                    
                    # Switch possession
                    possession = defense
                    field_position = 25
                    down = 1
                    distance = 10
                    
                    # Start new drive
                    current_drive = {
                        'team': possession,
                        'start_position': field_position,
                        'plays': [],
                        'result': None,
                        'quarter': quarter,
                        'time': clock_display
                    }
                    continue
                
                if down > 4:
                    # Turnover on downs
                    current_drive['result'] = 'TURNOVER ON DOWNS'
                    
                    # Complete the drive
                    current_drive['end_position'] = field_position
                    current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                    drive_summary.append(current_drive)
                    
                    # Print drive summary
                    print(f"\nDrive Summary - {possession} TURNOVER ON DOWNS!")
                    print(f"Started at: {current_drive['start_position']} yard line")
                    print(f"Plays: {len(current_drive['plays'])}")
                    print(f"Yards: {current_drive['yards']}")
                    print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                    print(f"Failed to convert: {down-1} & {distance + yards_gained}")
                    
                    # Switch possession
                    possession = defense
                    field_position = 100 - field_position  # Flip field
                    down = 1
                    distance = 10
                    
                    # Start new drive
                    current_drive = {
                        'team': possession,
                        'start_position': field_position,
                        'plays': [],
                        'result': None,
                        'quarter': quarter,
                        'time': clock_display
                    }
                    continue
            
            # Keep field position in bounds
            field_position = max(0, min(100, field_position))
            
            # Handle end of quarter
            if quarter_change:
                print(f"\nEnd of Quarter {quarter-1}")
                print(f"Score: {team1} {score[team1]}, {team2} {score[team2]}")
                
                # If it's halftime, add a special message
                if quarter == 3:
                    print("HALFTIME")
                    
                    # Complete the current drive and start a new one after halftime
                    if current_drive['plays']:
                        current_drive['result'] = 'END OF HALF'
                        current_drive['end_position'] = field_position
                        current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                        drive_summary.append(current_drive)
                        
                        # Start a new drive in the second half
                        current_drive = {
                            'team': possession,
                            'start_position': field_position,
                            'plays': [],
                            'result': None,
                            'quarter': quarter,
                            'time': clock_display
                        }
        
        # Game over
        elapsed = time.time() - start_time
        print(f"\nSimulation completed in {elapsed:.2f} seconds")
        
        # If there's an incomplete drive at the end of the game, record it
        if current_drive['plays']:
            current_drive['result'] = 'END OF GAME'
            current_drive['end_position'] = field_position
            current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
            drive_summary.append(current_drive)
        
        # Print final game statistics
        print("\n📊 Game Statistics:")
        print(f"{team1.ljust(4)} | {team2.ljust(4)}")
        print("-" * 30)
        print(f"Total Yards: {stats[team1]['total_yards']:.0f} | {stats[team2]['total_yards']:.0f}")
        print(f"Pass Yards: {stats[team1]['pass_yards']:.0f} | {stats[team2]['pass_yards']:.0f}")
        print(f"Rush Yards: {stats[team1]['rush_yards']:.0f} | {stats[team2]['rush_yards']:.0f}")
        print(f"Turnovers: {stats[team1]['turnovers']} | {stats[team2]['turnovers']}")
        print(f"Score: {score}")
        
        # Print drive summary
        print("\n🏈 Drive Summary:")
        
        team1_drives = [d for d in drive_summary if d['team'] == team1]
        team2_drives = [d for d in drive_summary if d['team'] == team2]
        
        print(f"\n{team1} Drives: {len(team1_drives)}")
        td_drives = len([d for d in team1_drives if d['result'] == 'TOUCHDOWN'])
        fg_drives = len([d for d in team1_drives if d['result'] == 'FIELD GOAL'])
        punt_drives = len([d for d in team1_drives if d['result'] == 'PUNT'])
        turnover_drives = len([d for d in team1_drives if d['result'] in ['TURNOVER', 'TURNOVER ON DOWNS']])
        
        print(f"  Touchdowns: {td_drives}")
        print(f"  Field Goals: {fg_drives}")
        print(f"  Punts: {punt_drives}")
        print(f"  Turnovers: {turnover_drives}")
        
        print(f"\n{team2} Drives: {len(team2_drives)}")
        td_drives = len([d for d in team2_drives if d['result'] == 'TOUCHDOWN'])
        fg_drives = len([d for d in team2_drives if d['result'] == 'FIELD GOAL'])
        punt_drives = len([d for d in team2_drives if d['result'] == 'PUNT'])
        turnover_drives = len([d for d in team2_drives if d['result'] in ['TURNOVER', 'TURNOVER ON DOWNS']])
        
        print(f"  Touchdowns: {td_drives}")
        print(f"  Field Goals: {fg_drives}")
        print(f"  Punts: {punt_drives}")
        print(f"  Turnovers: {turnover_drives}")


In [47]:
simulator = NFLSimulator()
simulator.simulate_game("MIA", "HOU")

Loading data files...
Simulator ready with 32 teams available.

Simulating MIA vs HOU...

Drive Summary - HOU PUNT!
Started at: 25 yard line
Plays: 4
Yards: 0.3442168393437619
Time: 60:00 in Q1
Punt distance: 36.4 yards

Drive Summary - MIA TOUCHDOWN!
Started at: 38.215265699458826 yard line
Plays: 6
Yards: 61.784734300541174
Time: 57:49 in Q1
Score: MIA 7, HOU 0

Drive Summary - HOU PUNT!
Started at: 25 yard line
Plays: 4
Yards: 2.732219105919242
Time: 54:16 in Q1
Punt distance: 36.5 yards

Drive Summary - MIA TURNOVER!
Started at: 35.757556309853896 yard line
Plays: 5
Yards: 26.841022743676646
Time: 52:22 in Q1
Final play: Punt play (no data)

Drive Summary - HOU PUNT!
Started at: 37.40142094646946 yard line
Plays: 4
Yards: 3.597275652496201
Time: 49:46 in Q1
Punt distance: 43.2 yards

End of Quarter 1
Score: MIA 7, HOU 0

Drive Summary - MIA FIELD GOAL!
Started at: 15.76363256804055 yard line
Plays: 15
Yards: 83.96224079165219
Time: 47:39 in Q1
Score: MIA 10, HOU 0

Drive Summary - 

SystemError: CPUDispatcher(<function run_numba_intensive_computation at 0x0000015DE4F13740>) returned a result with an exception set

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Rectangle
import time
import os

# Try to import advanced computation functions if available
try:
    import nfl_advanced_computation as adv
    HAS_ADVANCED_COMPUTATION = True
except ImportError:
    HAS_ADVANCED_COMPUTATION = False

# Global simulation settings
QUARTERS = 4
QUARTER_LENGTH = 15 * 60
COMPUTE_INTENSITY = 500

from numba import njit, prange

@njit(parallel=True, fastmath=True)
def rowwise_norm_parallel(mat):
    """Compute row-wise L2 norm using Numba with parallel loops"""
    nrows = mat.shape[0]
    result = np.empty(nrows)
    for i in prange(nrows):
        total = 0.0
        for j in range(mat.shape[1]):
            total += mat[i, j] ** 2
        result[i] = np.sqrt(total)
    return result

@njit(parallel=True, fastmath=True)
def run_numba_intensive_computation_parallel(iterations, a, b):
    for _ in range(iterations):
        c = a @ b

        # Parallel norm and sort
        d = rowwise_norm_parallel(c)
        e = np.sort(d)

        # Element-wise matrix ops
        f = np.exp(c[:50, :50])
        g = np.sin(c[50:100, 50:100])

        # Evolve matrices with noise
        noise = np.random.normal(0, 1, a.shape)
        a = 0.99 * a + 0.01 * noise
        b = 0.99 * b + 0.01 * noise

    return a, b

class NFLSimulator:
    def __init__(self, games_file="games.csv", plays_file="plays.csv"):
        print("Loading data files...")
        self.games_df = pd.read_csv(games_file)
        self.plays_df = pd.read_csv(plays_file)
        self.teams = np.unique(np.concatenate([
            self.games_df['homeTeamAbbr'].unique(), 
            self.games_df['visitorTeamAbbr'].unique()
        ]))
        self._preprocess_data()
        print(f"Simulator ready with {len(self.teams)} teams available.")

    def _preprocess_data(self):
        self.plays_df['playType'] = 'Other'
        if 'isDropback' in self.plays_df.columns:
            pass_mask = (self.plays_df['isDropback'] == 1) & (~self.plays_df['passResult'].isna())
            self.plays_df.loc[pass_mask, 'playType'] = 'Pass'
            run_mask = (self.plays_df['isDropback'] != 1) & (~self.plays_df['yardsGained'].isna())
            self.plays_df.loc[run_mask, 'playType'] = 'Run'
        if 'playDescription' in self.plays_df.columns:
            punt_mask = self.plays_df['playDescription'].str.contains('PUNT', na=False)
            fg_mask = self.plays_df['playDescription'].str.contains('FIELD GOAL', na=False)
            self.plays_df.loc[punt_mask, 'playType'] = 'Punt'
            self.plays_df.loc[fg_mask, 'playType'] = 'Field Goal'
        self.team_plays = {}
        for team in self.teams:
            team_df = self.plays_df[self.plays_df['possessionTeam'] == team]
            if len(team_df) > 0:
                self.team_plays[team] = {
                    'Pass': team_df[team_df['playType'] == 'Pass'],
                    'Run': team_df[team_df['playType'] == 'Run'],
                    'Punt': team_df[team_df['playType'] == 'Punt'],
                    'Field Goal': team_df[team_df['playType'] == 'Field Goal']
                }

    def _intensive_computation(self, iterations=100):
        a = np.random.normal(0, 1, (500, 500))
        b = np.random.normal(0, 1, (500, 500))
        run_numba_intensive_computation_parallel(iterations, a, b)

    def display_available_teams(self):
        teams = sorted(self.teams)
        print("\nAvailable Teams:")
        print(", ".join(teams))
        return teams

    def get_play(self, team, play_type, field_position, down, distance):
        """Get a play outcome based on historical data and intensive computation"""
        # Run computation to make this function take longer
        self._intensive_computation(COMPUTE_INTENSITY)
        
        # Use advanced computation if available
        if HAS_ADVANCED_COMPUTATION:
            # Create weather conditions (random for simulation)
            weather = {
                'wind_speed': np.random.randint(0, 20),
                'wind_direction': np.random.randint(0, 360),
                'temperature': np.random.randint(20, 95),
                'precipitation': np.random.uniform(0, 1.0),
                'field_condition': np.random.uniform(0.5, 1.0)
            }
            
            # Create game situation
            situation = {
                'down': down,
                'distance': distance,
                'field_position': field_position,
                'score_diff': 0  # Default neutral score
            }
            
            # Create team data (random for simulation)
            offense_team = {f"player_{i}": np.random.normal(0.7, 0.15, 10) for i in range(11)}
            defense_team = {f"player_{i}": np.random.normal(0.7, 0.15, 10) for i in range(11)}
            
            # Run advanced computations
            # 1. Simulate player tracking
            try:
                tracking_data = adv.simulate_player_tracking(22, 100)
                
                # Analyze tracking data to adjust play outcome
                # More computation to slow down simulation
                player_speeds = np.zeros(22)
                player_distances = np.zeros(22)
                for i in range(22):
                    # Calculate speeds and distances for each player
                    for t in range(1, 100):
                        # Calculate displacement between time steps
                        displacement = tracking_data[i, t, :] - tracking_data[i, t-1, :]
                        # Calculate speed
                        speed = np.linalg.norm(displacement) / 0.1  # 0.1 second time step
                        # Update maximum speed
                        player_speeds[i] = max(player_speeds[i], speed)
                        # Add to total distance
                        player_distances[i] += np.linalg.norm(displacement)
            except:
                # If tracking simulation fails, create dummy data
                tracking_data = None
                player_speeds = np.random.normal(5, 1, 22)
                player_distances = np.random.normal(20, 5, 22)
            
            # 2. Apply weather effects
            try:
                weather_adjustments = adv.simulate_weather_effects(field_position, play_type, weather)
            except:
                # If weather simulation fails, use default adjustments
                weather_adjustments = {
                    'yards': 0,
                    'accuracy': 0,
                    'turnover_risk': 0,
                    'success_probability': 0
                }
            
            # 3. Create player matchups
            try:
                matchup_results = adv.create_matchup_matrix(offense_team, defense_team, play_type, situation)
            except:
                # If matchup simulation fails, use default results
                matchup_results = {
                    'success_probability': 0.5,
                    'expected_yards': 5,
                    'turnover_risk': 0.03,
                    'advantage': 0
                }
                
            # Use advanced computations to determine play outcome
            base_yards = matchup_results['expected_yards']
            
            # Add randomness to yards
            yards = base_yards + np.random.normal(0, 3)
            
            # Apply weather adjustments
            yards += weather_adjustments['yards']
            
            # Adjust for field position (harder to gain yards near goal line)
            if field_position > 80:
                yards *= 0.7  # Reduce yards in red zone
            
            # Determine if turnover occurs
            turnover_probability = matchup_results['turnover_risk'] + weather_adjustments['turnover_risk']
            turnover = np.random.random() < turnover_probability
            
            # Determine if touchdown occurs
            if field_position + yards >= 100:
                touchdown = True
                yards = 100 - field_position  # Adjust yards to reach end zone
            else:
                touchdown = False
            
            # Determine if field goal is successful
            if play_type == 'Field Goal':
                fg_distance = 100 - field_position + 17  # Add 17 yards for end zone + snap
                base_fg_probability = 1.0 - (fg_distance / 65)  # Base probability by distance
                adjusted_fg_probability = base_fg_probability + weather_adjustments['success_probability']
                field_goal = np.random.random() < adjusted_fg_probability
            else:
                field_goal = False
            
            # Create play description
            if touchdown:
                description = f"{play_type} play for TOUCHDOWN!"
            elif field_goal:
                description = "FIELD GOAL IS GOOD"
            elif turnover:
                if play_type == 'Pass':
                    description = "Pass INTERCEPTED"
                else:
                    description = f"{play_type} play - FUMBLE"
            else:
                description = f"{play_type} play for {yards:.1f} yards"
            
            # Estimate play time (in seconds)
            play_time = np.random.randint(25, 40)
            
            return {
                'yards': yards,
                'time': play_time,
                'turnover': turnover,
                'touchdown': touchdown,
                'field_goal': field_goal,
                'description': description
            }
        
        # If no advanced computation, use the original method
        # If no data available for this team/play type, return default
        if team not in self.team_plays or play_type not in self.team_plays[team]:
            return {
                'yards': np.random.normal(5, 10),
                'time': np.random.randint(25, 40),
                'turnover': np.random.random() < 0.05,
                'touchdown': np.random.random() < 0.05,
                'field_goal': play_type == 'Field Goal' and np.random.random() < 0.7,
                'description': f"{play_type} play (simulated)"
            }
        
        # Get play data for this team/play type
        plays = self.team_plays[team][play_type]
        
        if len(plays) == 0:
            # No plays of this type, return default
            return {
                'yards': np.random.normal(5, 10),
                'time': np.random.randint(25, 40),
                'turnover': np.random.random() < 0.05,
                'touchdown': np.random.random() < 0.05,
                'field_goal': play_type == 'Field Goal' and np.random.random() < 0.7,
                'description': f"{play_type} play (no data)"
            }
        
        # Select a random play
        play = plays.sample(1).iloc[0]
        
        # Get yards gained (default to 0 if not available)
        yards = play['yardsGained'] if 'yardsGained' in play and not pd.isna(play['yardsGained']) else 0
        
        # Add some randomness to yards gained
        yards = yards + np.random.normal(0, yards/4 if yards > 0 else 2)
        
        # Check for touchdown based on field position
        touchdown = False
        if field_position + yards >= 100:
            touchdown = True
            yards = 100 - field_position
        
        # Check for turnover
        turnover = False
        if 'playDescription' in play:
            desc = str(play['playDescription']).upper()
            turnover = 'INTERCEPTION' in desc or 'FUMBLE' in desc
        
        # Check for field goal
        field_goal = False
        if play_type == 'Field Goal' and 'playDescription' in play:
            desc = str(play['playDescription']).upper()
            field_goal = 'FIELD GOAL IS GOOD' in desc
            
            # Adjust field goal probability based on field position
            if field_position >= 65:  # 35 yard line or closer
                field_goal = np.random.random() < 0.9  # 90% probability
            elif field_position >= 55:  # 45 yard line or closer
                field_goal = np.random.random() < 0.7  # 70% probability
            else:
                field_goal = np.random.random() < 0.4  # 40% probability
        
        # Create a play description
        if touchdown:
            description = f"{play_type} play for TOUCHDOWN!"
        elif field_goal:
            description = "FIELD GOAL IS GOOD"
        elif turnover:
            if play_type == 'Pass':
                description = "Pass INTERCEPTED"
            else:
                description = f"{play_type} play - FUMBLE"
        else:
            description = f"{play_type} play for {yards:.1f} yards"
        
        # Estimate play time
        play_time = np.random.randint(25, 40)
        
        return {
            'yards': yards,
            'time': play_time,
            'turnover': turnover,
            'touchdown': touchdown,
            'field_goal': field_goal,
            'description': description
        }
    
    def choose_play_type(self, down, distance, field_position, score_diff, time_remaining):
        """Choose play type based on game situation"""
        # Base probabilities
        pass_prob = 0.6
        run_prob = 0.4
        fg_prob = 0.0
        punt_prob = 0.0
        
        # Adjust based on down
        if down == 3:
            # More likely to pass on 3rd down
            pass_prob += 0.2
            run_prob -= 0.2
        elif down == 4:
            # 4th down decision making
            if field_position > 65:  # Inside opponent's 35
                # Field goal territory
                fg_prob = 0.8
                pass_prob = 0.1
                run_prob = 0.1
                punt_prob = 0.0
            elif field_position > 50:  # Between 50 and opponent's 35
                # Go for it or punt
                if distance <= 2:  # Short yardage
                    pass_prob = 0.3
                    run_prob = 0.4
                    punt_prob = 0.3
                else:
                    punt_prob = 0.9
                    pass_prob = 0.05
                    run_prob = 0.05
            else:  # Own territory
                # Likely punt
                punt_prob = 0.95
                pass_prob = 0.025
                run_prob = 0.025
        
        # Adjust for time remaining
        if time_remaining < 120:  # Last 2 minutes
            if score_diff < 0:  # Losing
                pass_prob += 0.2  # More passes when behind late
                run_prob -= 0.2
            elif score_diff > 14:  # Winning big
                run_prob += 0.2  # More runs when ahead late
                pass_prob -= 0.2
        
        # Normalize probabilities
        total = pass_prob + run_prob + fg_prob + punt_prob
        pass_prob /= total
        run_prob /= total
        fg_prob /= total
        punt_prob /= total
        
        # Choose play type
        r = np.random.random()
        if r < pass_prob:
            return 'Pass'
        elif r < pass_prob + run_prob:
            return 'Run'
        elif r < pass_prob + run_prob + fg_prob:
            return 'Field Goal'
        else:
            return 'Punt'
    
    def simulate_game(self, team1, team2):
        """Simulate a full game between two teams"""
        start_time = time.time()
        print(f"\nSimulating {team1} vs {team2}...")
        
        # Game state initialization
        score = {team1: 0, team2: 0}
        possession = np.random.choice([team1, team2])  # Random team starts with ball
        field_position = 25  # Start at own 25 yard line
        down = 1
        distance = 10
        time_remaining = QUARTERS * QUARTER_LENGTH
        quarter = 1
        
        # Game statistics
        stats = {
            team1: {'total_yards': 0, 'pass_yards': 0, 'rush_yards': 0, 'turnovers': 0},
            team2: {'total_yards': 0, 'pass_yards': 0, 'rush_yards': 0, 'turnovers': 0}
        }
        
        # Drive tracking
        drive_summary = []
        current_drive = {
            'team': possession,
            'start_position': field_position,
            'plays': [],
            'result': None,
            'quarter': quarter,
            'time': f"{time_remaining // 60:02d}:{time_remaining % 60:02d}"
        }
        
        # Team colors
        team_colors = {team1: 'red', team2: 'blue'}
        
        # Game loop
        while time_remaining > 0:
            # Get the defensive team
            defense = team2 if possession == team1 else team1
            
            # Calculate score differential from possession team perspective
            score_diff = score[possession] - score[defense]
            
            # Choose play type
            play_type = self.choose_play_type(
                down, distance, field_position, score_diff, time_remaining
            )
            
            # Get play result
            play = self.get_play(possession, play_type, field_position, down, distance)
            
            # Apply play result
            yards_gained = play['yards']
            play_time = play['time']
            
            # Update statistics
            stats[possession]['total_yards'] += yards_gained
            if play_type == 'Pass':
                stats[possession]['pass_yards'] += yards_gained
            elif play_type == 'Run':
                stats[possession]['rush_yards'] += yards_gained
            
            # Calculate time used
            time_remaining -= play_time
            
            # Update quarter if needed
            new_quarter = min(4, 1 + (QUARTERS * QUARTER_LENGTH - time_remaining) // QUARTER_LENGTH)
            quarter_change = new_quarter > quarter
            quarter = new_quarter
            
            # Format game clock
            minutes = time_remaining // 60
            seconds = time_remaining % 60
            clock_display = f"{int(minutes):02}:{int(seconds):02}"
            
            # Add play to current drive
            current_drive['plays'].append({
                'type': play_type,
                'yards': yards_gained,
                'description': play['description'],
                'down': down,
                'distance': distance,
                'field_position': field_position
            })
            
            # Handle scoring plays
            if play['touchdown']:
                score[possession] += 7  # TD + PAT for simplicity
                current_drive['result'] = 'TOUCHDOWN'
                
                # Complete the drive
                current_drive['end_position'] = 100
                current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                drive_summary.append(current_drive)
                
                # Print drive summary
                print(f"\nDrive Summary - {possession} TOUCHDOWN!")
                print(f"Started at: {current_drive['start_position']} yard line")
                print(f"Plays: {len(current_drive['plays'])}")
                print(f"Yards: {current_drive['yards']}")
                print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                print(f"Score: {possession} {score[possession]}, {defense} {score[defense]}")
                
                # Start new drive
                possession = defense
                field_position = 25
                down = 1
                distance = 10
                current_drive = {
                    'team': possession,
                    'start_position': field_position,
                    'plays': [],
                    'result': None,
                    'quarter': quarter,
                    'time': clock_display
                }
                continue
                
            elif play['field_goal']:
                score[possession] += 3
                current_drive['result'] = 'FIELD GOAL'
                
                # Complete the drive
                current_drive['end_position'] = field_position
                current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                drive_summary.append(current_drive)
                
                # Print drive summary
                print(f"\nDrive Summary - {possession} FIELD GOAL!")
                print(f"Started at: {current_drive['start_position']} yard line")
                print(f"Plays: {len(current_drive['plays'])}")
                print(f"Yards: {current_drive['yards']}")
                print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                print(f"Score: {possession} {score[possession]}, {defense} {score[defense]}")
                
                # Start new drive
                possession = defense
                field_position = 25
                down = 1
                distance = 10
                current_drive = {
                    'team': possession,
                    'start_position': field_position,
                    'plays': [],
                    'result': None,
                    'quarter': quarter,
                    'time': clock_display
                }
                continue
            
            # Handle turnovers
            if play['turnover']:
                stats[possession]['turnovers'] += 1
                current_drive['result'] = 'TURNOVER'
                
                # Complete the drive
                current_drive['end_position'] = field_position + yards_gained
                current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                drive_summary.append(current_drive)
                
                # Print drive summary
                print(f"\nDrive Summary - {possession} TURNOVER!")
                print(f"Started at: {current_drive['start_position']} yard line")
                print(f"Plays: {len(current_drive['plays'])}")
                print(f"Yards: {current_drive['yards']}")
                print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                print(f"Final play: {play['description']}")
                
                # Switch possession
                possession = defense
                field_position = 100 - (field_position + yards_gained)  # Flip field
                field_position = max(0, min(100, field_position))  # Keep in bounds
                down = 1
                distance = 10
                
                # Start new drive
                current_drive = {
                    'team': possession,
                    'start_position': field_position,
                    'plays': [],
                    'result': None,
                    'quarter': quarter,
                    'time': clock_display
                }
                continue
            
            # Handle punts
            if play_type == 'Punt':
                # Punt distance based on field position
                punt_distance = np.random.normal(45, 8)
                current_drive['result'] = 'PUNT'
                
                # Complete the drive
                current_drive['end_position'] = field_position
                current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                drive_summary.append(current_drive)
                
                # Print drive summary
                print(f"\nDrive Summary - {possession} PUNT!")
                print(f"Started at: {current_drive['start_position']} yard line")
                print(f"Plays: {len(current_drive['plays'])}")
                print(f"Yards: {current_drive['yards']}")
                print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                print(f"Punt distance: {punt_distance:.1f} yards")
                
                # Switch possession
                possession = defense
                field_position = max(0, min(100, 100 - (field_position + punt_distance)))
                down = 1
                distance = 10
                
                # Start new drive
                current_drive = {
                    'team': possession,
                    'start_position': field_position,
                    'plays': [],
                    'result': None,
                    'quarter': quarter,
                    'time': clock_display
                }
                continue
            
            # Update field position
            field_position += yards_gained
            
            # Check for first down
            if yards_gained >= distance:
                down = 1
                distance = 10
                if field_position > 100:
                    field_position = 100  # Cap at 100 (goal line)
            else:
                down += 1
                distance -= yards_gained
                if field_position > 100:
                    # Crossed goal line
                    score[possession] += 7  # TD + PAT
                    current_drive['result'] = 'TOUCHDOWN'
                    
                    # Complete the drive
                    current_drive['end_position'] = 100
                    current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                    drive_summary.append(current_drive)
                    
                    # Print drive summary
                    print(f"\nDrive Summary - {possession} TOUCHDOWN!")
                    print(f"Started at: {current_drive['start_position']} yard line")
                    print(f"Plays: {len(current_drive['plays'])}")
                    print(f"Yards: {current_drive['yards']}")
                    print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                    print(f"Score: {possession} {score[possession]}, {defense} {score[defense]}")
                    
                    # Switch possession
                    possession = defense
                    field_position = 25
                    down = 1
                    distance = 10
                    
                    # Start new drive
                    current_drive = {
                        'team': possession,
                        'start_position': field_position,
                        'plays': [],
                        'result': None,
                        'quarter': quarter,
                        'time': clock_display
                    }
                    continue
                
                if down > 4:
                    # Turnover on downs
                    current_drive['result'] = 'TURNOVER ON DOWNS'
                    
                    # Complete the drive
                    current_drive['end_position'] = field_position
                    current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                    drive_summary.append(current_drive)
                    
                    # Print drive summary
                    print(f"\nDrive Summary - {possession} TURNOVER ON DOWNS!")
                    print(f"Started at: {current_drive['start_position']} yard line")
                    print(f"Plays: {len(current_drive['plays'])}")
                    print(f"Yards: {current_drive['yards']}")
                    print(f"Time: {current_drive['time']} in Q{current_drive['quarter']}")
                    print(f"Failed to convert: {down-1} & {distance + yards_gained}")
                    
                    # Switch possession
                    possession = defense
                    field_position = 100 - field_position  # Flip field
                    down = 1
                    distance = 10
                    
                    # Start new drive
                    current_drive = {
                        'team': possession,
                        'start_position': field_position,
                        'plays': [],
                        'result': None,
                        'quarter': quarter,
                        'time': clock_display
                    }
                    continue
            
            # Keep field position in bounds
            field_position = max(0, min(100, field_position))
            
            # Handle end of quarter
            if quarter_change:
                print(f"\nEnd of Quarter {quarter-1}")
                print(f"Score: {team1} {score[team1]}, {team2} {score[team2]}")
                
                # If it's halftime, add a special message
                if quarter == 3:
                    print("HALFTIME")
                    
                    # Complete the current drive and start a new one after halftime
                    if current_drive['plays']:
                        current_drive['result'] = 'END OF HALF'
                        current_drive['end_position'] = field_position
                        current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
                        drive_summary.append(current_drive)
                        
                        # Start a new drive in the second half
                        current_drive = {
                            'team': possession,
                            'start_position': field_position,
                            'plays': [],
                            'result': None,
                            'quarter': quarter,
                            'time': clock_display
                        }
        
        # Game over
        elapsed = time.time() - start_time
        print(f"\nSimulation completed in {elapsed:.2f} seconds")
        
        # If there's an incomplete drive at the end of the game, record it
        if current_drive['plays']:
            current_drive['result'] = 'END OF GAME'
            current_drive['end_position'] = field_position
            current_drive['yards'] = current_drive['end_position'] - current_drive['start_position']
            drive_summary.append(current_drive)
        
        # Print final game statistics
        print("\n📊 Game Statistics:")
        print(f"{team1.ljust(4)} | {team2.ljust(4)}")
        print("-" * 30)
        print(f"Total Yards: {stats[team1]['total_yards']:.0f} | {stats[team2]['total_yards']:.0f}")
        print(f"Pass Yards: {stats[team1]['pass_yards']:.0f} | {stats[team2]['pass_yards']:.0f}")
        print(f"Rush Yards: {stats[team1]['rush_yards']:.0f} | {stats[team2]['rush_yards']:.0f}")
        print(f"Turnovers: {stats[team1]['turnovers']} | {stats[team2]['turnovers']}")
        print(f"Score: {score}")
        
        # Print drive summary
        print("\n🏈 Drive Summary:")
        
        team1_drives = [d for d in drive_summary if d['team'] == team1]
        team2_drives = [d for d in drive_summary if d['team'] == team2]
        
        print(f"\n{team1} Drives: {len(team1_drives)}")
        td_drives = len([d for d in team1_drives if d['result'] == 'TOUCHDOWN'])
        fg_drives = len([d for d in team1_drives if d['result'] == 'FIELD GOAL'])
        punt_drives = len([d for d in team1_drives if d['result'] == 'PUNT'])
        turnover_drives = len([d for d in team1_drives if d['result'] in ['TURNOVER', 'TURNOVER ON DOWNS']])
        
        print(f"  Touchdowns: {td_drives}")
        print(f"  Field Goals: {fg_drives}")
        print(f"  Punts: {punt_drives}")
        print(f"  Turnovers: {turnover_drives}")
        
        print(f"\n{team2} Drives: {len(team2_drives)}")
        td_drives = len([d for d in team2_drives if d['result'] == 'TOUCHDOWN'])
        fg_drives = len([d for d in team2_drives if d['result'] == 'FIELD GOAL'])
        punt_drives = len([d for d in team2_drives if d['result'] == 'PUNT'])
        turnover_drives = len([d for d in team2_drives if d['result'] in ['TURNOVER', 'TURNOVER ON DOWNS']])
        
        print(f"  Touchdowns: {td_drives}")
        print(f"  Field Goals: {fg_drives}")
        print(f"  Punts: {punt_drives}")
        print(f"  Turnovers: {turnover_drives}")

In [45]:
simulator = NFLSimulator()
#simulator.display_available_teams()
simulator.simulate_game("MIN","IND")

Loading data files...
Simulator ready with 32 teams available.

Simulating MIN vs IND...

Drive Summary - IND PUNT!
Started at: 25 yard line
Plays: 4
Yards: 5.620302795340084
Time: 60:00 in Q1
Punt distance: 46.4 yards

Drive Summary - MIN FIELD GOAL!
Started at: 23.005739369372222 yard line
Plays: 12
Yards: 44.6508237030818
Time: 58:08 in Q1
Score: MIN 3, IND 0

Drive Summary - IND PUNT!
Started at: 25 yard line
Plays: 4
Yards: 3.479545675119706
Time: 52:00 in Q1
Punt distance: 31.4 yards

Drive Summary - MIN PUNT!
Started at: 40.13545802724502 yard line
Plays: 4
Yards: -0.1338618974868666
Time: 49:46 in Q1
Punt distance: 41.6 yards

Drive Summary - IND PUNT!
Started at: 18.35005759525646 yard line
Plays: 4
Yards: 6.4970584700790255
Time: 47:35 in Q1
Punt distance: 45.7 yards

End of Quarter 1
Score: MIN 3, IND 0

Drive Summary - MIN PUNT!
Started at: 29.425607593077714 yard line
Plays: 4
Yards: 8.417094280432366
Time: 45:30 in Q1
Punt distance: 43.7 yards

Drive Summary - IND PUNT!
S